In [17]:
#-*- coding:utf-8

import os
# 정규 표현식 라이브러리
import re

# scikit-learn 라이브러리
from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 형태소 분석기 라이브러리
from konlpy.tag import Hannanum
from konlpy.tag import Kkma

# pandas 라이브러리 임포트
import pandas as pd

import numpy as np

In [18]:
# 데이터 정리
dir_prefix = "./data/hkib-20000-40075/"
target_dir = "HKIB-20000"
cat_dirs = ["health", "economy", "science", "education", "culture", "society", "industry", "leisure", "politics"]
cat_prefixs = ["건강", "경제", "과학", "교육", "문화", "사회", "산업", "여가", "정치"]

files = os.listdir(dir_prefix + target_dir)

In [24]:
# 5분할된 텍스트 파일을 각각 처리
for file in files:
    # 데이터가 담긴 파일만 처리
    if not file.endswith(".txt"):
        continue
    
    # 각 텍스트 파일을 처리
    with open(dir_prefix + target_dir + "/" + file, encoding="utf8") as currfile:
        doc_cnt = 0
        docs = []
        curr_doc = None

        # 기사 단위로 분할해서 리스트를 생성
        for curr_line in currfile:
            if curr_line.startswith("@DOCUMENT"):
                if curr_doc is not None:
                    # 새로운 @DOCUMENT를 만났을 때, docs에 문서 추가
                    docs.append(curr_doc)

                curr_doc = curr_line
                doc_cnt += 1
                continue
            curr_doc += curr_line

        # 각 기사를 대주제별로 분류해서 기사별 파일로 정리
        for doc in docs:
            doc_lines = doc.split("\n")
            doc_no = doc_lines[1][9:]

            # 주제 추출
            doc_cat03 = ""
            for line in doc_lines[:10]:
                if line.startswith("#CAT'03:"):
                    doc_cat03 = line[10:]
                    break

            # 추출한 주제별로 디렉터리 정리
            for cat_prefix in cat_prefixs:
                if doc_cat03.startswith(cat_prefix):
                    dir_index = cat_prefixs.index(cat_prefix)
                    break

            # 문서 정보를 제거하고 기사 본문만 남기기
            filtered_lines = []
            for line in doc_lines:
                if not (line.startswith("#") or line.startswith("@")):
                    filtered_lines.append(line)

            # 주제별 디렉터리에 기사를 파일로 쓰기
            filename = "hkib-" + doc_no + ".txt"
            filepath = dir_prefix + target_dir + "/" + cat_dirs[dir_index]

            if not os.path.exists(filepath):
                os.makedirs(filepath)

            f = open(filepath + "/" + filename, "w", encoding="utf8")
            f.write("\n".join(filtered_lines))
            f.close()

In [34]:
# 대상시 되는 주제 폴더 선택(교육, 건강)
dirs = ["education", "health"]

# 각 폴더의 파일을 하나씩 읽어 들임
for i, d in enumerate(dirs):
    # 파일 목록 읽어오기
    files = os.listdir(dir_prefix + target_dir + "/" + d)

    for file in files:
        # 각 파일을 읽어 들이기
        f = open(dir_prefix + target_dir + "/" + d + "/" + file, "r", encoding="utf8")
        raw = f.read()

        # 정규표현식을 사용해 불필요한 문자열을 제거한 다음 파일 내용을 출력
        reg_raw = re.sub(r"[0-9a-zA-Z]", "", raw)
        reg_raw = re.sub(r"[-'@#:/◆◇!-\"*\(\)]", "", raw)
        reg_raw = re.sub(r"[ ]+", " ", reg_raw)
        reg_raw = reg_raw.replace("\n", " ")

        # 파일 닫기
        f.close()

In [35]:
# 대상이 되는 주제 폴더 선택(교육, 건강)
dirs = ["education", "health"]

# 기사에 출현하는 단어와 레이블을 저장할 리스트를 생성
# 설명변수
x_ls = []
# 목적변수
y_ls = []

tmp1 = []
tmp2 = ""

# 형태소 분석기 객체 생성
tokenizer = Kkma()

# 각 폴더의 파일을 하나씩 읽어 들이며, 전처리 후 리스트에 저장
for i, d in enumerate(dirs):
    # 파일 목록 읽어오기
    files = os.listdir(dir_prefix + target_dir + "/" + d)

    for file in files:
        # 각 파일을 읽어 들이기
        f = open(dir_prefix + target_dir + "/" + d + "/" + file, "r", encoding="utf8")
        raw = f.read()

        # 정규표현식을 사용해 불필요한 문자열을 제거한 다음 파일 내용을 출력
        reg_raw = re.sub(r"[0-9a-zA-Z]", "", raw)
        reg_raw = re.sub(r"[-'@#:/◆◇!-\"*\(\)]", "", raw)
        reg_raw = re.sub(r"[ ]+", " ", reg_raw)
        reg_raw = reg_raw.replace("\n", " ")

        # 형태소 분석을 거쳐 명사만 추출한 리스트를 생성
        tokens = tokenizer.nouns(reg_raw)

        for token in tokens:
            tmp1.append(token)

        tmp2 = " ".join(tmp1)
        x_ls.append(tmp2)
        tmp1 = []

        # 기사 주제 레이블을 리스트에 저장
        y_ls.append(i)

        # 파일 닫기
        f.close()

In [36]:
# 데이터프레임으로 변환해서 설명변수를 화면에 출력
pd.DataFrame(x_ls)

,0
0,통제 대학 대학정원 정원 교육 교육여건 여건 연동 자율화 올 올안 안 안이 확정 9...
1,본고사 본고사실시 실시 고등학생 간 과외 과외열풍 열풍 가운데 고교생 월 평균 과외...
2,97 97년 년 대학 절반 절반이상 이상 자율 정원 정원책정권 책 정권 정원자율화 ...
3,90 90년가을 년 가을 미국 미국여행중 여행 중 사립대 사립대총장 총장 나 대화 ...
4,국민학교 정규 영어 영어교육 교육 실시 모양 학교 특별 특별활동시간 활동 시간 영어...
...,...
996,밀레니엄 베이비 하늘 세요 199912162018 10 병원 2 우려 제왕절개 사양...
997,신애 신애양 양 성탄절 199912241930 1 1224 1930 199912 0...
998,외국계 외국계제약사 제약사 신약 신약시장 시장 점령 199912270929 10 국...
999,밀레니엄 베이비 모유 먹 먹입시다 입 시다 199912271914 10 정부 유니 ...


In [37]:
# 첫 번째 기사로부터 추출한 단어를 출력
print(x_ls[0])

통제 대학 대학정원 정원 교육 교육여건 여건 연동 자율화 올 올안 안 안이 확정 96 96년 년 시행 본격적 자율 경쟁 경쟁시대 시대 눈앞 감회 이 대학조직도 조직도 입학 입학관리 관리 학무 취업 취업관리등 등 학생 지원 위주 위주체제 체제 뿐 교수 교수평가 평가 저 도입 적극 권장 이행 여부 정부 재정 재정지원 차 차등화 등화 개혁 개혁방향 방향 교육수요 수요 폭발적 증가 안주 내실 외적 성장 하향 하향평준화 평준화 우리 기회 부담 동시 오늘 대학교육 국제화 국제화시대 가열 가열해진 해진 고급 전문 전문인력 인력 배출 내 연구 봉사 봉사등 기능 기능수행 수행 획기적 개선 수 수월성추구 월성 추구 국가적 요구 때 간섭 나 창의 창의속 속 책무 민주적 분위기 조성 필수 필수불가결하 불가 결하 책임 학교 학교간의 간의 노력 실적 도태 수 실력 실력제일주의시대 제일주의 도래 예고 선 교육내실 과언 지원자 때문 정원자율화추진 추진 사정 이제 시작 2 2천년대 천 년대 진입 평균 평균대입경쟁률 대입 경쟁률 1 1대1 대 전망 결국 시설 시설등 유인 정도 성사 정부당국 당국 재정난 강화 무리 정원자율화 보증 엄격 기준 기준마련 마련 정확 과거 대학정원령 정 원령 실사 책정 입시 입시관리 부정 물론 교육당국 하루아침 감독 감독능력 능력 학부모 걱정 의미 대 대학과 학과 엊그제 교정 최루탄 학사 학사업무 업무 타율 굴레 발전 궤도 모두 라도


In [38]:
# 목적변수를 화면에 출력
print(y_ls)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [39]:
# 설명변수와 목적변수를 NumPy 배열로 변환
x_array = np.array(x_ls)
y_array = np.array(y_ls)

# 단어 출현 횟수를 계수
cntvec = CountVectorizer()
x_cntvecs = cntvec.fit_transform(x_array)
x_cntarray = x_cntvecs.toarray()

# 데이터프레임으로 단어 출현 횟수 출력
pd.DataFrame(x_cntarray)

,0,1,2,3,4,5,6,7,8,9,...,38174,38175,38176,38177,38178,38179,38180,38181,38182,38183
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# 단어와 단어의 인덱스 표시
for k, v in sorted(cntvec.vocabulary_.items(), key=lambda x:x[1]):
    print(k, v)

268
한옥집 36269
한원 36270
한의 36271
한의과대 36272
한의과대학 36273
한의대 36274
한의대가 36275
한의대간 36276
한의대병원등 36277
한의대생 36278
한의사 36279
한의사가 36280
한의사도 36281
한의사의 36282
한의사자격 36283
한의사측 36284
한의사협회 36285
한의사협회측 36286
한의업사 36287
한의업자 36288
한의예 36289
한의예과 36290
한의예과중 36291
한의원 36292
한의쪽 36293
한의학 36294
한의학과 36295
한이 36296
한인 36297
한인학생 36298
한일 36299
한일경제협상 36300
한일뿐 36301
한일약품 36302
한일은행 36303
한일장신 36304
한일합방 36305
한일합섬 36306
한자 36307
한자교육 36308
한자말 36309
한자읽기 36310
한자하이쿠 36311
한재 36312
한재훈씨23 36313
한재훈씨24 36314
한전 36315
한정 36316
한조 36317
한족 36318
한족학생 36319
한중 36320
한중간 36321
한중관계가 36322
한중교류증 36323
한중수교 36324
한진 36325
한진그룹 36326
한진실업학교 36327
한진해운 36328
한쪽 36329
한차례 36330
한참 36331
한창 36332
한청광박사 36333
한총련 36334
한탄 36335
한탄강 36336
한파 36337
한편 36338
한편식당 36339
한풀이 36340
한풀이식 36341
한학 36342
한해 36343
한해동안 36344
한화 36345
한화등 36346
한흑갈등 36347
할당 36348
할당제 36349
할당제쿼터제 36350
할머니 36351
할머니75 36352
할머님 36353
할부 36354
할아버지 36355
할아버지99 36356
할애 36357
할인 36358
함경도 36359
함량 36360
함를 36361
함부 36362
함수 36363
함수관계 

In [41]:
# 단어의 TF-IDF 계산
tfidf_vec = TfidfVectorizer(use_idf=True)
x_tfidf_vecs = tfidf_vec.fit_transform(x_array)
x_tfidf_array = x_tfidf_vecs.toarray()

# 데이터프레임으로 변환해서 단어의 출현 횟수를 출력
pd.DataFrame(x_tfidf_array)

,0,1,2,3,4,5,6,7,8,9,...,38174,38175,38176,38177,38178,38179,38180,38181,38182,38183
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.063306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# 데이터를 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(x_tfidf_array, y_array, test_size=0.2)

# 데이터 건수 확인
print(len(train_X))
print(len(test_X))

800
201


In [43]:
# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [44]:
# 훈련 데이터 텐서 생성
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 생성
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

torch.Size([800, 38184])
torch.Size([800])


In [45]:
# 설명변수와 목적변수의 텐서를 합친다
train = TensorDataset(train_X, train_Y)

# 첫 번쨰 텐서 내용 확인
print(train[0])

# 미니배치 분할
train_loader = DataLoader(train, batch_size=100, shuffle=True)

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor(0))


In [51]:
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(38184, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x)

# 인스턴스 생성
model = Net()

In [53]:
# 오차함수
criterion = nn.CrossEntropyLoss()

# 최적화 기법 선택
optimizer = optim.Adam(model.parameters(), lr=0.005)

# 학습
for eopch in range(20):
    total_loss = 0

    # 분할한 데이터(미니배치)를 차례로 꺼내옴
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x)

        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.data

    print(eopch+1, total_loss)

1tensor(3.9998)
2tensor(0.6559)
3tensor(0.2750)
4tensor(0.0689)
5tensor(0.0599)
6tensor(0.0604)
7tensor(0.0529)
8tensor(0.0516)
9tensor(0.0462)
10tensor(0.0448)
11tensor(0.0438)
12tensor(0.0433)
13tensor(0.0428)
14tensor(0.0425)
15tensor(0.0423)
16tensor(0.0423)
17tensor(0.0420)
18tensor(0.0420)
19tensor(0.0419)
20tensor(0.0419)


In [55]:
# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 혹은 1이 되도록
result = torch.max(model(test_x).data, 1)[1]

# 모델의 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 정확도 출력
accuracy

0.9751243781094527

In [57]:
test_X[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])